In [1]:
# Загрузка необходимых библиотек
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Устанавливаем устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Параметры
BATCH_SIZE = 16384
model_path = '2.2_DCN_MLP.pth'
test_path = 'fv2_test.parquet'  # Путь к тестовым данным
custom_data_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/custom_data/'
data_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/data/'
test_output_path = '2.2_predictions.csv' 

In [4]:
test = pd.read_parquet(f'{custom_data_folder}{test_path}', engine='pyarrow')

In [5]:
# Загрузка метаинформации
items_meta = pd.read_parquet(f'{data_folder}items_meta.parquet', engine='pyarrow')
items_meta['item_id'] = items_meta['item_id'].astype('category')
items_meta.set_index('item_id', inplace=True)

In [6]:
# Преобразуем embeddings в массив и создаем индексы
item_embeddings_array = torch.tensor(
    np.stack(items_meta['embeddings'].values),
    device=device,
    dtype=torch.float32
)
item_id_to_index = {item: idx for idx, item in enumerate(items_meta.index)}

In [7]:
class TestDataset(Dataset):
    def __init__(self, interactions, device):
        self.device = device
        self.users = torch.tensor(interactions['user_id'].values, dtype=torch.long, device=self.device)
        self.items = torch.tensor(interactions['item_id'].values, dtype=torch.long, device=self.device)
        self.ages = torch.tensor(interactions['age'].values, dtype=torch.long, device=self.device)
        self.item_durations = torch.tensor(interactions['item_duration'].values, dtype=torch.long, device=self.device)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ages[idx], self.item_durations[idx]


# Создаем DataLoader для тестовых данных
test_ds = TestDataset(test, device)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [8]:
class DCN(nn.Module):
    def __init__(self, input_dim, num_cross_layers):
        super(DCN, self).__init__()
        self.input_dim = input_dim
        self.num_cross_layers = num_cross_layers
        self.cross_weights = nn.ParameterList(
            [nn.Parameter(torch.randn(input_dim, 1)) for _ in range(num_cross_layers)]
        )
        self.cross_biases = nn.ParameterList(
            [nn.Parameter(torch.randn(input_dim)) for _ in range(num_cross_layers)]
        )

    def forward(self, x):
        x0 = x
        for i in range(self.num_cross_layers):
            x = x0 * (x @ self.cross_weights[i]) + self.cross_biases[i] + x
        return x


class DCNWithMLP(nn.Module):
    def __init__(self, input_dim, num_cross_layers=3, hidden_dim=128, output_dim=3):
        super(DCNWithMLP, self).__init__()
        self.batch_norm = nn.BatchNorm1d(input_dim)
        self.dcn = DCN(input_dim, num_cross_layers)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 64)
        self.fc3 = nn.Linear(64, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.batch_norm(x)
        x = self.dcn(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
# Параметры модели
input_dim = len(test.columns) + 32 # Включаем embeddings
num_cross_layers = 3

In [10]:
# Инициализация модели и загрузка весов
model = DCNWithMLP(input_dim, num_cross_layers).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

C:\Users\Николай\AppData\Local\Temp\ipykernel_15372\3681416054.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


DCNWithMLP(
  (batch_norm): BatchNorm1d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dcn): DCN(
    (cross_weights): ParameterList(
        (0): Parameter containing: [torch.float32 of size 36x1 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 36x1 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 36x1 (cuda:0)]
    )
    (cross_biases): ParameterList(
        (0): Parameter containing: [torch.float32 of size 36 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 36 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 36 (cuda:0)]
    )
  )
  (fc1): Linear(in_features=36, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=3, bias=True)
  (relu): ReLU()
)

In [12]:
from tqdm import tqdm

all_predictions = []

with torch.no_grad():
    for users, items, ages, item_durations in tqdm(test_dl, desc="Inference Progress", unit="batch"):
        indices = [item_id_to_index[item.item()] for item in items]
        embeddings = item_embeddings_array[indices]
        
        inputs = torch.cat((
            users.unsqueeze(1),
            items.unsqueeze(1),
            ages.unsqueeze(1),
            item_durations.unsqueeze(1),
            embeddings
        ), dim=1).float()
        
        outputs = model(inputs)  # Вероятности или сырые logits для каждого класса
        
        # Приводим logits к вероятностям (если это необходимо)
        probabilities = F.softmax(outputs, dim=1)
        
        # Веса для классов: 0, 1, 2
        class_weights = torch.tensor([0, 1, 2], device=probabilities.device, dtype=probabilities.dtype)
        
        # Рассчитываем взвешенное значение
        weighted_predictions = torch.sum(probabilities * class_weights, dim=1).cpu().numpy()
        
        all_predictions.extend(weighted_predictions)

Inference Progress: 100%|██████████| 102/102 [00:49<00:00,  2.05batch/s]


In [13]:
# Сохранение предсказаний
test['predict'] = all_predictions
test.drop(columns=['age', 'item_duration'], inplace=True)
test[['user_id', 'item_id', 'predict']].to_csv(f'{test_output_path}', index=False)
print(f"Предсказания сохранены в {test_output_path}")

Предсказания сохранены в 2.2_predictions.csv
